# Monitoring Data Updates

## Congestion Data Update
* Data is from RITIS (ritis.org), which in turn uses INRIX data. Anyone at TRPA can get an account, but you need to sign a data form to get full access to all of the data. We're downloading the performance summaries for the segments/dates in question and pulling out the bidirectional travel times.

In [1]:
# import packages and setup notebook
import arcpy
from datetime import datetime
import os
import sys
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
import sqlalchemy as sa
import pandas as pd
from arcgis.features import FeatureSet, GeoAccessor, GeoSeriesAccessor
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# # Set Pandas display options to show all rows and columns
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# in memory output file path
wk_memory = "memory" + "\\"

# set workspace and sde connections 
working_folder = "C:\GIS"
workspace      = "C:\GIS\Scratch.gdb"
arcpy.env.workspace = "C:\GIS\Scratch.gdb"

# network path to connection files
filePath = "C:\\GIS\\DB_CONNECT"

# database file path 
sdeBase    = os.path.join(filePath, "Vector.sde")
sdeCollect = os.path.join(filePath, "Collection.sde")

travelTimeTable = os.path.join(sdeBase, "sde.SDE.Travel_Times")

# setup connection string
connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=sql12;DATABASE=sde;UID=sde;PWD=staff"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)

with engine.begin() as sdeConnect:
    dfTTsde      = pd.read_sql("SELECT * FROM sde.SDE.Travel_Times", sdeConnect)

In [2]:
dfTTsde.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455 entries, 0 to 454
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   OBJECTID          455 non-null    int64  
 1   Name              455 non-null    object 
 2   Season            455 non-null    object 
 3   Year              442 non-null    float64
 4   Value             433 non-null    float64
 5   GlobalID          455 non-null    object 
 6   created_user      0 non-null      object 
 7   created_date      0 non-null      object 
 8   last_edited_user  0 non-null      object 
 9   last_edited_date  0 non-null      object 
dtypes: float64(2), int64(1), object(7)
memory usage: 35.7+ KB


In [4]:
dfTTsde.to_csv(os.path.join(workspace,"TravelTime_Staged.csv"))

In [3]:
# staging data
stagingFC     = "F:\GIS\PROJECTS\Transportation\Modeling and Monitoring\Data\TravelTimes.gdb\Roads_JSCopy"
# Convert the feature set to a Pandas DataFrame
df = pd.DataFrame.spatial.from_featureclass(stagingFC)
df.columns.to_list()


['OBJECTID',
 'NAME',
 'Shape_Leng',
 'GlobalID',
 'created_user',
 'created_date',
 'last_edited_user',
 'last_edited_date',
 'TT_Spring_22',
 'TT_Summer_22',
 'TT_Fall_22',
 'TT_Winter_2122',
 'TT_21',
 'TT_Spring_21',
 'TT_Summer_21',
 'TT_Fall_21',
 'TT_Winter_20to21',
 'TT_20',
 'TT_Spring_20',
 'TT_Summer_20',
 'TT_Fall_20',
 'TT_Winter_1920',
 'TT_19',
 'TT_Spring_19',
 'TT_Summer_19',
 'TT_Fall_19',
 'TT_Winter_1819',
 'TT_18',
 'TT_Spring_18',
 'TT_Summer_18',
 'TT_Fall_18',
 'TT_Winter_1718',
 'TT_17',
 'TT_Spring_17',
 'TT_Summer_17',
 'TT_Fall_17',
 'TT_Winter_1617',
 'TT_16',
 'TT_Spring_16',
 'TT_Summer_16',
 'TT_Fall_16',
 'TT_Winter_1516',
 'TT_15',
 'TT_22',
 'SpeedLimitTT',
 'SHAPE']

In [5]:
# Function to extract the second item or handle the case with only one "_"
def extract_middle_item(value):
    parts = value.split('_')
    
    if len(parts) == 2:
        return "All"
    elif len(parts) > 2:
        return parts[1]
    else:
        return None

# Specify columns to drop by name
columns_to_drop = ['OBJECTID', 
                    'Shape_Leng',
                    'GlobalID',
                    'created_user',
                    'created_date',
                    'last_edited_user',
                    'last_edited_date',
                    'SpeedLimitTT',
                    'SHAPE']

# Drop the specified columns
dfNew = df.drop(columns=columns_to_drop)

# Specify the columns to keep 
columns_to_keep = ['NAME']

# Melt the DataFrame by specifying the 'id_vars' parameter
melted_df = pd.melt(dfNew, id_vars=columns_to_keep, var_name='TravelSeason', value_name='Value')

dfTT = melted_df

dfTT['Year']   = ("20"+dfTT['TravelSeason'].str[-2:]).astype(int)

# Apply the function to create a new field 'NewField'
dfTT['Season'] = dfTT['TravelSeason'].apply(extract_middle_item)
# Rename the column
dfTT = dfTT.rename(columns={'NAME': 'Name'})
# Define the new column order
new_column_order = ['Name', 'Season', 'Year', 'Value']

# Set the new order and index for the columns
dfFinal = dfTT[new_column_order]
dfFinal


,Name,Season,Year,Value
0,Nevada SR 431,Spring,2022,15.115000
1,Nevada SR 28 (West of Incline),Spring,2022,8.445000
2,California SR 267,Spring,2022,4.095000
3,SR 89 (North of Tahoe City),Spring,2022,16.709999
4,California State Route 28,Spring,2022,18.305000
...,...,...,...,...
463,SR 207 Kingsbury Grade,All,2022,5.300000
464,Pioneer Trail,All,2022,12.515000
465,US 50 (Y to State Line),All,2022,9.955000
466,US 50 (Echo to Y),All,2022,13.000000


In [22]:
arcpy.ListFields("Congestion_Staging")

[<Field object at 0x1e762a47760[0x1e76420e030]>,
 <Field object at 0x1e762a47970[0x1e76420ed10]>,
 <Field object at 0x1e762a47730[0x1e76420eed0]>,
 <Field object at 0x1e762a47d30[0x1e76420ee70]>,
 <Field object at 0x1e762a47ee0[0x1e76420edf0]>,
 <Field object at 0x1e762a47cd0[0x1e76420eb50]>,
 <Field object at 0x1e762a47610[0x1e76420ef70]>]

In [25]:
# List of field name changes (old_field_name: new_field_name)
field_name_changes = {
    'name'  : 'Name',
    'season': 'Season',
    'year'  : 'Year',
    'value' : 'Value'
}

fc = os.path.join(workspace,"Congestion_Staging")

# Loop through the field name changes and rename the fields
for old_field_name, new_field_name in field_name_changes.items():
    arcpy.management.AlterField(fc, old_field_name, new_field_name, new_field_name)

print("Field names have been altered successfully.")

Field names have been altered successfully.


In [26]:
# replaces features in outfc with exact same schema
def updateSDE(inputfc,outfc, fieldnames):
        # disconnect all users
    print("\nDisconnecting all users...")
    arcpy.DisconnectUser(sdeBase, "ALL")

    print ("Unregistering feature dataset as versioned...")
    # unregister the sde feature class as versioned
    arcpy.UnregisterAsVersioned_management(outfc,"NO_KEEP_EDIT","COMPRESS_DEFAULT")
    print ("Finished unregistering feature dataset as versioned.")
    # set overwrite files envrionment setting to True
    arcpy.env.overwriteOutput = True
    # deletes all rows from the SDE feature class
    arcpy.TruncateTable_management(outfc)
    print ("\nDeleted all records in: {}\n".format(outfc))

    # insert rows from Temporary feature class to SDE feature class
    with arcpy.da.InsertCursor(outfc, fieldnames) as oCursor:
        count = 0
        with arcpy.da.SearchCursor(inputfc, fieldnames) as iCursor:
            for row in iCursor:
                oCursor.insertRow(row)
                count += 1
                if count % 100 == 0:
                    print("Inserting record {0} into SDE table".format(count))

    # disconnect all users
    print("\nDisconnecting all users...")
    arcpy.DisconnectUser(sdeBase, "ALL")
    print("\nRegistering feature dataset as versioned...")
    # register SDE feature class as versioned
    arcpy.RegisterAsVersioned_management(outfc, "NO_EDITS_TO_BASE")
    print("\nFinished registering feature dataset as versioned.")
    # confirm feature class was created
    print("\nUpdated " + outfc)

# # Convert the Pandas DataFrame to a NumPy structured array
# dfFinal.spatial.to_table(location= os.path.join(workspace, "Congestion_Staging"))
inputTable = "Congestion_Staging"
field_names = ['Name', 'Season', 'Year', 'Value']

updateSDE(inputTable, travelTimeTable, field_names)


Disconnecting all users...
Unregistering feature dataset as versioned...
Finished unregistering feature dataset as versioned.

Deleted all records in: C:\GIS\DB_CONNECT\Vector.sde\sde.SDE.Travel_Times


Disconnecting all users...

Registering feature dataset as versioned...

Finished registering feature dataset as versioned.

Updated C:\GIS\DB_CONNECT\Vector.sde\sde.SDE.Travel_Times


In [ ]:
arcpy.management.Append(
    inputs=r"C:\GIS\Scratch.gdb\Congestion_Staging",
    target=r"C:\Users\mbindl\Documents\ArcGIS\Projects\Vector.sde\sde.SDE.Travel_Times",
    schema_type="NO_TEST",
    field_mapping=r'Name "Name" true true false 255 Text 0 0,First,#,C:\GIS\Scratch.gdb\Congestion_Staging,name,0,60;Season "Season" true true false 255 Text 0 0,First,#,C:\GIS\Scratch.gdb\Congestion_Staging,season,0,12;Year "Year" true true false 2 Short 0 5,First,#,C:\GIS\Scratch.gdb\Congestion_Staging,year,-1,-1;Value "Value" true true false 8 Double 8 38,First,#,C:\GIS\Scratch.gdb\Congestion_Staging,value,-1,-1;GlobalID "GlobalID" false false true 38 GlobalID 0 0,First,#',
    subtype="",
    expression="",
    match_fields=None,
    update_geometry="NOT_UPDATE_GEOMETRY"
)

## Crash Data Update

In [1]:
## Setup 
# import modules
import pandas as pd
import os
import pyodbc
from datetime import datetime
import arcpy
import numpy as np
import urllib
#import geojson
#import json
#from urllib.request import urlopen
from arcgis.features import FeatureSet, GeoAccessor, GeoSeriesAccessor
#Test 2
# setup workspace folder
#workspace = "//Trpa-fs01/gis/PROJECTS/ResearchAnalysis/Monitoring/Data/Crash/"
workspace = "F:/gis/PROJECTS/ResearchAnalysis/Monitoring/Data/Crash/"

# setup environment variables
arcpy.env.overwriteOutput = True
#arcpy.env.workspace = "//Trpa-fs01/GIS/PROJECTS/ResearchAnalysis/Monitoring/Data/Crash/CrashData/CrashData.gdb"
arcpy.env.workspace = "F:/GIS/PROJECTS/ResearchAnalysis/Monitoring/Data/Crash/CrashData/CrashData.gdb"

# create a spatial reference object for the output coordinate system 
# output projection for data going into SDE should be UTM Zone 10N (EPSG: 26910)
out_coordinate_system = arcpy.SpatialReference(26910)

# network path to connection files
#filePath = "//Trpa-fs01/GIS/DB_CONNECT"
filePath = "F:/GIS/DB_CONNECT"

# database file path 
sdeBase  = os.path.join(filePath, "Vector.sde")

# SDE feature classes needed for spatial joins
corridor = os.path.join(sdeBase, 'sde.SDE.Transportation\sde.SDE.Corridor')
trpa     = os.path.join(sdeBase, 'sde.SDE.Jurisdictions\sde.SDE.TRPA_bdy')

# define csv lat/long field names for xy table to point
x_coords = 'POINT_X'
y_coords = 'POINT_Y'

# SDE feature class to update
crashSDE  = os.path.join(sdeBase, 'sde.SDE.Transportation\sde.SDE.Highway_Collisions')

# Get Crash Data
caCrashes = os.path.join(workspace, "BothCACounties_Crashes_1321_Unclean.csv")
dfCACrash = pd.read_csv(caCrashes)

nvCrashes = os.path.join(workspace, "NV_LAKE TAHOE BASIN - ALL ROADS 2013-2020.csv")
dfNVCrash = pd.read_csv(nvCrashes)

nvCrashes21 = os.path.join(workspace, "[INSERT 2021 CRASH FILE]")
dfNVCrash21 = pd.read_csv(nvCrashes21)
    
# # in memory files
memory = "memory" + "\\"

# replaces features in outfc with exact same schema
def updateSDE(inputfc,outfc, fieldnames):
        # disconnect all users
    print("\nDisconnecting all users...")
    arcpy.DisconnectUser(sde, "ALL")

    # deletes all rows from the SDE feature class
    arcpy.TruncateTable_management(outfc)
    print ("\nDeleted all records in: {}\n".format(outfc))

    # insert rows from Temporary feature class to SDE feature class
    with arcpy.da.InsertCursor(outfc, fieldnames) as oCursor:
        count = 0
        with arcpy.da.SearchCursor(inputfc, fieldnames) as iCursor:
            for row in iCursor:
                oCursor.insertRow(row)
                count += 1
                if count % 1000 == 0:
                    print("Inserting record {0} into SDE table".format(count))

    # disconnect all users
    print("\nDisconnecting all users...")
    arcpy.DisconnectUser(sde, "ALL")
    # confirm feature class was created
    print("\nUpdated " + outfc)
    
# function to move spatial join data to parcel master staging
def fieldJoinCalc(updateFC, updateFieldsList, sourceFC, sourceFieldsList):
    from time import strftime  
    print ("Started data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))
#     log.info("Started data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))
    # Use list comprehension to build a dictionary from arcpy SearchCursor  
    valueDict = {r[0]:(r[1:]) for r in arcpy.da.SearchCursor(sourceFC, sourceFieldsList)}  
   
    with arcpy.da.UpdateCursor(updateFC, updateFieldsList) as updateRows:  
        for updateRow in updateRows:  
            # store the Join value of the row being updated in a keyValue variable  
            keyValue = updateRow[0]  
            # verify that the keyValue is in the Dictionary  
            if keyValue in valueDict:  
                # transfer the value stored under the keyValue from the dictionary to the updated field.  
                updateRow[1] = valueDict[keyValue][0]  
                updateRows.updateRow(updateRow)    
    del valueDict  
    print ("Finished data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))
#     log.info("Finished data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))


C:\Users\jschmid\AppData\Local\Temp\ipykernel_26132\1404647701.py:48: DtypeWarning: Columns (6,17) have mixed types. Specify dtype option on import or set low_memory=False.
  dfCACrash = pd.read_csv(caCrashes)


In [2]:

## CA data translation
# convert date/time and case info
#convert state/county/city, year/date/time
dfCACrash['State']       = "CA"
dfCACrash['City']        = dfCACrash['CITY']
dfCACrash['County']      = dfCACrash['COUNTY']
dfCACrash['Year']        = dfCACrash['ACCIDENT_YEAR']
dfCACrash['Date']        = dfCACrash['COLLISION_DATE']
dfCACrash['4DigTime']    = dfCACrash['COLLISION_TIME'].astype(str).str.zfill(4)
dfCACrash['Hour']        = dfCACrash['4DigTime'].str[:2]
dfCACrash['Min']         = dfCACrash['4DigTime'].str[2:]
dfCACrash['Time']        = dfCACrash['Hour']+":"+dfCACrash['Min']
dfCACrash['Data_Source'] = "CHP/SWITRS"

#Convert severity
dfCACrash.loc[dfCACrash['COLLISION_SEVERITY'].isin([1]),  'Crash_Severity'] = 'Fatal'
dfCACrash.loc[dfCACrash['COLLISION_SEVERITY'].isin([2]),  'Crash_Severity'] = 'Severe injury'
dfCACrash.loc[dfCACrash['COLLISION_SEVERITY'].isin([3]),  'Crash_Severity'] = 'Other visible injury'
dfCACrash.loc[dfCACrash['COLLISION_SEVERITY'].isin([4]),  'Crash_Severity'] = 'Complaint of pain'
dfCACrash.loc[dfCACrash['COLLISION_SEVERITY'].isin([0]),  'Crash_Severity'] = 'Property damage only'
#Convert crash type
dfCACrash.loc[dfCACrash['TYPE_OF_COLLISION'].isin(["A"]),  'Crash_Type'] = 'Head-on'
dfCACrash.loc[dfCACrash['TYPE_OF_COLLISION'].isin(["B"]),  'Crash_Type'] = 'Sideswipe'
dfCACrash.loc[dfCACrash['TYPE_OF_COLLISION'].isin(["C"]),  'Crash_Type'] = 'Rear end'
dfCACrash.loc[dfCACrash['TYPE_OF_COLLISION'].isin(["D"]),  'Crash_Type'] = 'Angle-broadside'
dfCACrash.loc[dfCACrash['TYPE_OF_COLLISION'].isin(["E"]),  'Crash_Type'] = 'Hit object'
dfCACrash.loc[dfCACrash['TYPE_OF_COLLISION'].isin(["F"]),  'Crash_Type'] = 'Overturned'
dfCACrash.loc[dfCACrash['TYPE_OF_COLLISION'].isin(["G"]),  'Crash_Type'] = 'Vehicle/pedestrian'
dfCACrash.loc[dfCACrash['TYPE_OF_COLLISION'].isin(["H"]),  'Crash_Type'] = 'Other'
#convert lighting
dfCACrash.loc[dfCACrash['LIGHTING'].isin(["A"]),  'Lighting'] = "Daylight"
dfCACrash.loc[dfCACrash['LIGHTING'].isin(["B"]),  'Lighting'] = "Dusk - Dawn"
dfCACrash.loc[dfCACrash['LIGHTING'].isin(["C"]),  'Lighting'] = "Dark - Street Lights"
dfCACrash.loc[dfCACrash['LIGHTING'].isin(["D"]),  'Lighting'] = "Dark - No Street Lights"
dfCACrash.loc[dfCACrash['LIGHTING'].isin(["E"]),  'Lighting'] = "Dark - Street Lights Not Functioning"
dfCACrash.loc[dfCACrash['LIGHTING'].isin(["-"]),  'Lighting'] = "Not Stated"
#Convert weather 1
dfCACrash.loc[dfCACrash['WEATHER_1'].isin(["A"]),  'Weather_1'] = "Clear"
dfCACrash.loc[dfCACrash['WEATHER_1'].isin(["B"]),  'Weather_1'] = "Cloudy"
dfCACrash.loc[dfCACrash['WEATHER_1'].isin(["C"]),  'Weather_1'] = "Raining"
dfCACrash.loc[dfCACrash['WEATHER_1'].isin(["D"]),  'Weather_1'] = "Snowing"
dfCACrash.loc[dfCACrash['WEATHER_1'].isin(["E"]),  'Weather_1'] = "Fog"
dfCACrash.loc[dfCACrash['WEATHER_1'].isin(["F"]),  'Weather_1'] = "Other"
dfCACrash.loc[dfCACrash['WEATHER_1'].isin(["G"]),  'Weather_1'] = "Windy"
dfCACrash.loc[dfCACrash['WEATHER_1'].isin(["-"]),  'Weather_1'] = "Not Stated"
# convert weather 2
dfCACrash.loc[dfCACrash['WEATHER_2'].isin(["A"]),  'Weather_2'] = "Clear"
dfCACrash.loc[dfCACrash['WEATHER_2'].isin(["B"]),  'Weather_2'] = "Cloudy"
dfCACrash.loc[dfCACrash['WEATHER_2'].isin(["C"]),  'Weather_2'] = "Raining"
dfCACrash.loc[dfCACrash['WEATHER_2'].isin(["D"]),  'Weather_2'] = "Snowing"
dfCACrash.loc[dfCACrash['WEATHER_2'].isin(["E"]),  'Weather_2'] = "Fog"
dfCACrash.loc[dfCACrash['WEATHER_2'].isin(["F"]),  'Weather_2'] = "Other"
dfCACrash.loc[dfCACrash['WEATHER_2'].isin(["G"]),  'Weather_2'] = "Windy"
dfCACrash.loc[dfCACrash['WEATHER_2'].isin(["-"]),  'Weather_2'] = "Not Stated"
#Convert violation
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["0"]),  'Violation'] = "Unknown"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["1"]),  'Violation'] = "Driving/Biking Under the Influence"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["2"]),  'Violation'] = "Impeding Traffic"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["3"]),  'Violation'] = "Unsafe Speed"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["4"]),  'Violation'] = "Following Too Closely"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["5"]),  'Violation'] = "Wrong Side of Road"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["6"]),  'Violation'] = "Improper Passing"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["7"]),  'Violation'] = "Unsafe Lane Change"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["8"]),  'Violation'] = "Improper Turning"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["9"]),  'Violation'] = "Auotomobile Right of Way"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["10"]),  'Violation'] = "Pedestrian Right of Way"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["11"]),  'Violation'] = "Pedestrian Violation"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["12"]),  'Violation'] = "Traffic Signals and Signs"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["13"]),  'Violation'] = "Hazardous Parking"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["17"]),  'Violation'] = "Other Hazardous Violation"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["18"]),  'Violation'] = "Other Than Driver or Pedestrian"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["21"]),  'Violation'] = "Unsafe Backing or Starting"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["22"]),  'Violation'] = "Other Improper Driving"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["-"]),  'Violation'] = "Not Stated"
#Convert road surface
dfCACrash.loc[dfCACrash['ROAD_SURFACE'].isin(["A"]),  'Road_Surface'] = "Dry"
dfCACrash.loc[dfCACrash['ROAD_SURFACE'].isin(["B"]),  'Road_Surface'] = "Wet"
dfCACrash.loc[dfCACrash['ROAD_SURFACE'].isin(["C"]),  'Road_Surface'] = "Snowy or Icy"
dfCACrash.loc[dfCACrash['ROAD_SURFACE'].isin(["D"]),  'Road_Surface'] = "Slippery (Muddy, Oily, etc.)"
dfCACrash.loc[dfCACrash['ROAD_SURFACE'].isin(["-"]),  'Road_Surface'] = "Not Stated"
#Convert road condition 1
dfCACrash.loc[dfCACrash['ROAD_COND_1'].isin(["A"]),  'Road_Condition_1'] = "Holes, Deep Ruts"
dfCACrash.loc[dfCACrash['ROAD_COND_1'].isin(["B"]),  'Road_Condition_1'] = "Loose Material on Roadway"
dfCACrash.loc[dfCACrash['ROAD_COND_1'].isin(["C"]),  'Road_Condition_1'] = "Obstruction on Roadway"
dfCACrash.loc[dfCACrash['ROAD_COND_1'].isin(["D"]),  'Road_Condition_1'] = "Construction or Repair Zone"
dfCACrash.loc[dfCACrash['ROAD_COND_1'].isin(["E"]),  'Road_Condition_1'] = "Reduced Roadway Width"
dfCACrash.loc[dfCACrash['ROAD_COND_1'].isin(["F"]),  'Road_Condition_1'] = "Flooded"
dfCACrash.loc[dfCACrash['ROAD_COND_1'].isin(["G"]),  'Road_Condition_1'] = "Other"
dfCACrash.loc[dfCACrash['ROAD_COND_1'].isin(["H"]),  'Road_Condition_1'] = "No Unusual Condition"
dfCACrash.loc[dfCACrash['ROAD_COND_1'].isin(["-"]),  'Road_Condition_1'] = "Not Stated"
# conver road condition 2
dfCACrash.loc[dfCACrash['ROAD_COND_2'].isin(["A"]),  'Road_Condition_2'] = "Holes, Deep Ruts"
dfCACrash.loc[dfCACrash['ROAD_COND_2'].isin(["B"]),  'Road_Condition_2'] = "Loose Material on Roadway"
dfCACrash.loc[dfCACrash['ROAD_COND_2'].isin(["C"]),  'Road_Condition_2'] = "Obstruction on Roadway"
dfCACrash.loc[dfCACrash['ROAD_COND_2'].isin(["D"]),  'Road_Condition_2'] = "Construction or Repair Zone"
dfCACrash.loc[dfCACrash['ROAD_COND_2'].isin(["E"]),  'Road_Condition_2'] = "Reduced Roadway Width"
dfCACrash.loc[dfCACrash['ROAD_COND_2'].isin(["F"]),  'Road_Condition_2'] = "Flooded"
dfCACrash.loc[dfCACrash['ROAD_COND_2'].isin(["G"]),  'Road_Condition_2'] = "Other"
dfCACrash.loc[dfCACrash['ROAD_COND_2'].isin(["H"]),  'Road_Condition_2'] = "No Unusual Condition"
dfCACrash.loc[dfCACrash['ROAD_COND_2'].isin(["-"]),  'Road_Condition_2'] = "Not Stated"
#Convert pedestrian action
dfCACrash.loc[dfCACrash['PED_ACTION'].isin(["A"]),  'Pedestrian_Action'] = "No Pedestrian Involved"
dfCACrash.loc[dfCACrash['PED_ACTION'].isin(["B"]),  'Pedestrian_Action'] = "Crossing in Crosswalk at Intersection"
dfCACrash.loc[dfCACrash['PED_ACTION'].isin(["C"]),  'Pedestrian_Action'] = "Crossing in Crosswalk Not at Intersection"
dfCACrash.loc[dfCACrash['PED_ACTION'].isin(["D"]),  'Pedestrian_Action'] = "Crossing Not in Crosswalk"
dfCACrash.loc[dfCACrash['PED_ACTION'].isin(["E"]),  'Pedestrian_Action'] = "In Road, Including Shoulder"
dfCACrash.loc[dfCACrash['PED_ACTION'].isin(["F"]),  'Pedestrian_Action'] = "Not in Road"
dfCACrash.loc[dfCACrash['PED_ACTION'].isin(["G"]),  'Pedestrian_Action'] = "Approaching/Leaving School Bus"
dfCACrash.loc[dfCACrash['PED_ACTION'].isin(["-"]),  'Pedestrian_Action'] = "Not Stated"
#Convert hit and run
dfCACrash.loc[dfCACrash['HIT_AND_RUN'].isin(["F"]), 'Hit_and_Run'] = "Felony"
dfCACrash.loc[dfCACrash['HIT_AND_RUN'].isin(["M"]), 'Hit_and_Run'] = "Misdemeanor"
dfCACrash.loc[dfCACrash['HIT_AND_RUN'].isin(["N"]), 'Hit_and_Run'] = "Not Hit and Run"
#Convert MVIW
dfCACrash.loc[dfCACrash['MVIW'].isin(["A"]), 'Motor_Vehicle_Interacted_With'] = "Non-Collision"
dfCACrash.loc[dfCACrash['MVIW'].isin(["B"]), 'Motor_Vehicle_Interacted_With'] = "Pedestrian"
dfCACrash.loc[dfCACrash['MVIW'].isin(["C"]), 'Motor_Vehicle_Interacted_With'] = "Other Motor Vehicle"
dfCACrash.loc[dfCACrash['MVIW'].isin(["D"]), 'Motor_Vehicle_Interacted_With'] = "Motor Vehicle on Other Roadway"
dfCACrash.loc[dfCACrash['MVIW'].isin(["E"]), 'Motor_Vehicle_Interacted_With'] = "Parked Motor Vehicle"
dfCACrash.loc[dfCACrash['MVIW'].isin(["F"]), 'Motor_Vehicle_Interacted_With'] = "Train"
dfCACrash.loc[dfCACrash['MVIW'].isin(["G"]), 'Motor_Vehicle_Interacted_With'] = "Bicycle"
dfCACrash.loc[dfCACrash['MVIW'].isin(["H"]), 'Motor_Vehicle_Interacted_With'] = "Animal"
dfCACrash.loc[dfCACrash['MVIW'].isin(["I"]), 'Motor_Vehicle_Interacted_With'] = "Fixed Object"
dfCACrash.loc[dfCACrash['MVIW'].isin(["J"]), 'Motor_Vehicle_Interacted_With'] = "Other Object"
dfCACrash.loc[dfCACrash['MVIW'].isin(["0"]), 'Motor_Vehicle_Interacted_With'] = "Non-Collision and Additional Object"
dfCACrash.loc[dfCACrash['MVIW'].isin(["1"]), 'Motor_Vehicle_Interacted_With'] = "Pedestrian and Additional Object"
dfCACrash.loc[dfCACrash['MVIW'].isin(["2"]), 'Motor_Vehicle_Interacted_With'] = "Other Motor Vehicle and Additional Object"
dfCACrash.loc[dfCACrash['MVIW'].isin(["3"]), 'Motor_Vehicle_Interacted_With'] = "Motor Vehicle on Other Roadway and Additional Object"
dfCACrash.loc[dfCACrash['MVIW'].isin(["4"]), 'Motor_Vehicle_Interacted_With'] = "Parked Motor Vehicle and Additional Object"
dfCACrash.loc[dfCACrash['MVIW'].isin(["5"]), 'Motor_Vehicle_Interacted_With'] = "Train and Additional Object"
dfCACrash.loc[dfCACrash['MVIW'].isin(["6"]), 'Motor_Vehicle_Interacted_With'] = "Bicycle and Additional Object"
dfCACrash.loc[dfCACrash['MVIW'].isin(["7"]), 'Motor_Vehicle_Interacted_With'] = "Animal and Additional Object"
dfCACrash.loc[dfCACrash['MVIW'].isin(["8"]), 'Motor_Vehicle_Interacted_With'] = "Fixed Object and Additional Object"
dfCACrash.loc[dfCACrash['MVIW'].isin(["9"]), 'Motor_Vehicle_Interacted_With'] = "Other Object and Additional Object"
dfCACrash.loc[dfCACrash['MVIW'].isin(["-"]), 'Motor_Vehicle_Interacted_With'] = "Not Stated"
#convert case ID
dfCACrash['CA_Case_ID']            = dfCACrash['CASE_ID']
dfCACrash['NV_Accident_Num']       = np.nan
dfCACrash['NV_Accident_Rec_Num']   = np.nan
#Convert number injured/killed
dfCACrash['Num_Killed']            = dfCACrash['NUMBER_KILLED']
dfCACrash['Num_Injured']           = dfCACrash['NUMBER_INJURED']
dfCACrash['Num_Ped_Killed']        = dfCACrash['COUNT_PED_KILLED']
dfCACrash['Num_Ped_Injured']       = dfCACrash['COUNT_PED_INJURED']
dfCACrash['Num_Bicyclist_Killed']  = dfCACrash['COUNT_BICYCLIST_KILLED']
dfCACrash['Num_Bicyclist_Injured'] = dfCACrash['COUNT_BICYCLIST_INJURED']
dfCACrash['Num_Motorcyclist_Killed']  = dfCACrash['COUNT_MC_KILLED']
dfCACrash['Num_Motorcyclist_Injured'] = dfCACrash['COUNT_MC_INJURED']
print(dfCACrash['Num_Motorcyclist_Killed'])
#Convert number vehicles/parties
dfCACrash['Num_Vehicles']          = np.nan
dfCACrash['Num_Parties']           = dfCACrash['PARTY_COUNT']
#Convert alcohol/bike/ped involvement
dfCACrash['Alcohol_Involved']      = dfCACrash['ALCOHOL_INVOLVED']
dfCACrash['Pedestrian_Involved']   = dfCACrash['PEDESTRIAN_ACCIDENT']
dfCACrash['Bicycle_Involved']      = dfCACrash['BICYCLE_ACCIDENT']
dfCACrash['Motorcycle_Involved']   = dfCACrash['MOTORCYCLE_ACCIDENT']
dfCACrash['Corridor_ID']           = np.nan

# final list of fields
dfCACrash = dfCACrash[['State',
           'CA_Case_ID',
           'NV_Accident_Num',
           'NV_Accident_Rec_Num',
           'Corridor_ID',
           'County',
           'City',
           'Year',
           'Date',
           'Time',
           'Weather_1',
           'Weather_2',
           'Crash_Severity',
           'Num_Killed',
           'Num_Injured',
           'Num_Ped_Killed',
           'Num_Ped_Injured',
           'Num_Bicyclist_Killed',
           'Num_Bicyclist_Injured',
           'Num_Motorcyclist_Killed',
           'Num_Motorcyclist_Injured',
           'Crash_Type',
           'Num_Vehicles',
           'Num_Parties',
           'Violation',
           'Hit_and_Run',
           'Motor_Vehicle_Interacted_With',
           'Pedestrian_Action', 
           'Road_Condition_1',
           'Road_Condition_2',
           'Road_Surface',
           'Lighting',
           'Pedestrian_Involved',
           'Bicycle_Involved',
           'Motorcycle_Involved',
           'Alcohol_Involved',
           'Data_Source',
           'POINT_X',
           'POINT_Y']].copy()


0        0
1        0
2        0
3        0
4        0
        ..
10902    0
10903    0
10904    0
10905    0
10906    0
Name: Num_Motorcyclist_Killed, Length: 10907, dtype: int64


In [3]:

## NV Data Transformation
# set fields for time and case info
dfNVCrash['CA_Case_ID']           = np.nan
dfNVCrash['NV_Accident_Num']      = dfNVCrash['NV Accident Num']
dfNVCrash['NV_Accident_Rec_Num']  = dfNVCrash['NV Accident Rec Num']
dfNVCrash['City']                 = np.nan
dfNVCrash['Year']                 = dfNVCrash['Collision_Year']
dfNVCrash['Date']                 = dfNVCrash['Collision_Date']
dfNVCrash['Time']                 = dfNVCrash['Collision_Time']
dfNVCrash['Num_Vehicles']         = dfNVCrash['Total Vehicles']
dfNVCrash['Num_Parties']          = np.nan
dfNVCrash['Data_Source']          = "NDOT"

# Convert NV crash type and severity
dfNVCrash['Crash_Severity']       = dfNVCrash['COLLISION_SEVERITY']
dfNVCrash.loc[dfNVCrash['Crash Type'].isin(["ANGLE"]),  'Crash_Type']         = 'Angle-broadside'
dfNVCrash.loc[dfNVCrash['Crash Type'].isin(["BACKING"]),  'Crash_Type']       = 'Backing'
dfNVCrash.loc[dfNVCrash['Crash Type'].isin(["HEAD-ON"]),  'Crash_Type']       = 'Head-on'
dfNVCrash.loc[dfNVCrash['Crash Type'].isin(["NON-COLLISION"]),  'Crash_Type'] = 'Non-collision'
dfNVCrash.loc[dfNVCrash['Crash Type'].isin(["REAR-END"]),  'Crash_Type']      = 'Rear end'
dfNVCrash.loc[dfNVCrash['Crash Type'].isin(["REAR-TO-REAR"]),  'Crash_Type']  = 'Other'
dfNVCrash.loc[dfNVCrash['Crash Type'].isin(["SIDESWIPE, MEETING"]),  'Crash_Type'] = 'Sideswipe'
dfNVCrash.loc[dfNVCrash['Crash Type'].isin(["SIDESWIPE, OVERTAKING"]),  'Crash_Type'] = 'Sideswipe'
dfNVCrash.loc[dfNVCrash['Crash Type'].isin(["UNKNOWN"]),  'Crash_Type'] = 'Unknown'
#Convert # injured/killed
dfNVCrash['Num_Killed']   = dfNVCrash['Fatalities']
dfNVCrash['Num_Killed'].fillna(0)
dfNVCrash['Num_Injured'] = dfNVCrash['Injured']
dfNVCrash['Num_Injured'].fillna(0)
dfNVCrash['Num_Ped_Killed'] = np.nan
dfNVCrash['Num_Ped_Injured'] = np.nan
dfNVCrash['Num_Bicyclist_Killed'] = np.nan
dfNVCrash['Num_Bicyclist_Injured'] = np.nan
dfNVCrash['Num_Motorcyclist_Killed'] = np.nan
dfNVCrash['Num_Motorcyclist_Injured'] = np.nan
# convert crash info
dfNVCrash['Violation']  = "N/A"
dfNVCrash['Hit_and_Run'] = "N/A"
dfNVCrash['Motor_Vehicle_Interacted_With'] = "N/A"
dfNVCrash['Pedestrian_Action'] = "N/A"
# Process lighting
dfNVCrash['Lighting'] = dfNVCrash['LIGHTING']
dfNVCrash.loc[dfNVCrash['Lighting'].isin(["Dusk-Dawn"]),  'Lighting'] = "Dusk - Dawn"
dfNVCrash.loc[dfNVCrash['Lighting'].isin(["Dusk-dawn"]),  'Lighting'] = "Dusk - Dawn"
dfNVCrash.loc[dfNVCrash['Lighting'].isin(["Dark - street lights"]),  'Lighting'] = "Dark - Street Lights"
dfNVCrash.loc[dfNVCrash['Lighting'].isin(["Dark - no street lights"]),  'Lighting'] = "Dark - No Street Lights"
dfNVCrash.loc[dfNVCrash['Lighting'].isin(["Dark - Unknown Lighting"]),  'Lighting'] = "Dark - Unknown Lighting"
dfNVCrash.loc[dfNVCrash['Lighting'].isin(["Not stated"]),  'Lighting'] = "Not Stated"
#Process alcohol involvement (check outputs)
dfNVCrash['V1 Driver Factors'] = dfNVCrash['V1 Driver Factors'].fillna("Not stated")
dfNVCrash['V2 Driver Factors'] = dfNVCrash['V2 Driver Factors'].fillna("Not stated")
dfNVCrash.loc[dfNVCrash['V1 Driver Factors'].str.contains("DRINKING"), 'Alcohol_Involved'] = "Y"
dfNVCrash.loc[dfNVCrash['V2 Driver Factors'].str.contains("DRINKING"), 'Alcohol_Involved'] = "Y"
#Process bike/ped involvement (check outputs)
dfNVCrash['V1 All Events'] = dfNVCrash['V1 All Events'].fillna("Not stated")
dfNVCrash['V2 All Events'] = dfNVCrash['V2 All Events'].fillna("Not stated")
dfNVCrash.loc[dfNVCrash['V1 All Events'].str.contains("PEDESTRIAN"), 'Pedestrian_Involved'] = "Y"
dfNVCrash.loc[dfNVCrash['V2 All Events'].str.contains("PEDESTRIAN"), 'Pedestrian_Involved'] = "Y"
dfNVCrash.loc[dfNVCrash['V1 All Events'].str.contains("PEDAL CYCLE"), 'Bicycle_Involved'] = "Y"
dfNVCrash.loc[dfNVCrash['V2 All Events'].str.contains("PEDAL CYCLE"), 'Bicycle_Involved'] = "Y"
#Process motorcycle involvement (check outputs)
dfNVCrash['V1 Type'] = dfNVCrash['V1 Type'].fillna("Not stated")
dfNVCrash['V2 Type'] = dfNVCrash['V1 Type'].fillna("Not stated")
dfNVCrash.loc[dfNVCrash['V1 Type'].str.contains("MOTORCYCLE"), 'Motorcycle_Involved'] = "Y"
dfNVCrash.loc[dfNVCrash['V1 Type'].str.contains("MOTORBIKE"), 'Motorcycle_Involved'] = "Y"
dfNVCrash.loc[dfNVCrash['V1 Type'].str.contains("MOPED"), 'Motorcycle_Involved'] = "Y"
dfNVCrash.loc[dfNVCrash['V2 Type'].str.contains("MOTORCYCLE"), 'Motorcycle_Involved'] = "Y"
dfNVCrash.loc[dfNVCrash['V2 Type'].str.contains("MOTORBIKE"), 'Motorcycle_Involved'] = "Y"
dfNVCrash.loc[dfNVCrash['V2 Type'].str.contains("MOPED"), 'Motorcycle_Involved'] = "Y"
# #Convert road surface
dfNVCrash['Factors Roadway'] = dfNVCrash['Factors Roadway'].fillna("Not stated")
dfNVCrash['HWY Factors'] = dfNVCrash['HWY Factors'].fillna("Not stated")
dfNVCrash.loc[dfNVCrash['Factors Roadway'].str.contains("DRY"), 'Road_Surface'] = "Dry"
dfNVCrash.loc[dfNVCrash['Factors Roadway'].str.contains("WET"), 'Road_Surface'] = "Wet"
dfNVCrash.loc[dfNVCrash['Factors Roadway'].str.contains("WATER"), 'Road_Surface'] = "Wet"
dfNVCrash.loc[dfNVCrash['Factors Roadway'].str.contains("ICE"), 'Road_Surface'] = "Snowy or Icy"
dfNVCrash.loc[dfNVCrash['Factors Roadway'].str.contains("SNOW"), 'Road_Surface'] = "Snowy or Icy"
dfNVCrash.loc[dfNVCrash['Factors Roadway'].str.contains("SLUSH"), 'Road_Surface'] = "Snowy or Icy"
dfNVCrash.loc[dfNVCrash['Factors Roadway'].str.contains("OTHER"), 'Road_Surface'] = "Other"
dfNVCrash.loc[dfNVCrash['Factors Roadway'].str.contains("NA"), 'Road_Surface'] = "Not stated"
dfNVCrash.loc[dfNVCrash['Factors Roadway'].str.contains("UNKNOWN"), 'Road_Surface'] = "Unknown"
dfNVCrash.loc[dfNVCrash['Factors Roadway'].str.contains("Not stated"), 'Road_Surface'] = "Not stated"
#Convert road condition
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("NONE"), 'Road_Condition_1'] = "No Unusual Condition"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("UNKNOWN"), 'Road_Condition_1'] = "Unknown"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("WET, ICY, SNOW, SLUSH"), 'Road_Condition_1'] = "Weather"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("WEATHER"), 'Road_Condition_1'] = "Weather"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("BACKUP"), 'Road_Condition_1'] = "Other"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("GLARE"), 'Road_Condition_1'] = "Other"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("OTHER"), 'Road_Condition_1'] = "Other"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("VISUAL OBSTRUCTION"), 'Road_Condition_1'] = "Other"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("ROAD OBSTRUCTION"), 'Road_Condition_1'] = "Obstruction on Roadway"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("DEBRIS"), 'Road_Condition_1'] = "Loose Material on Roadway"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("WORK ZONE"), 'Road_Condition_1'] = "Construction or Repair Zone"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("ANIMAL"), 'Road_Condition_1'] = "Wildlife"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("RUTS"), 'Road_Condition_1'] = "Holes, Deep Ruts"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("Not stated"), 'Road_Condition_1'] = "Not Stated"
dfNVCrash['Road_Condition_2'] = np.nan

#Rename to match CA data
dfNVCrash['Weather_1']   = dfNVCrash['Weather']
dfNVCrash['Weather_2']   = np.nan
dfNVCrash['POINT_X']     = dfNVCrash['X']
dfNVCrash['POINT_Y']     = dfNVCrash['Y']
dfNVCrash['Corridor_ID'] = np.nan

# final list of fields
dfNVCrash = dfNVCrash[['State',
           'CA_Case_ID',
           'NV_Accident_Num',
           'NV_Accident_Rec_Num',
           'Corridor_ID',
           'County',
           'City',
           'Year',
           'Date',
           'Time',
           'Weather_1',
           'Weather_2',
           'Crash_Severity',
           'Num_Killed',
           'Num_Injured',
           'Num_Ped_Killed',
           'Num_Ped_Injured',
           'Num_Bicyclist_Killed',
           'Num_Bicyclist_Injured',
           'Crash_Type',
           'Num_Vehicles',
           'Num_Parties',
           'Violation',
           'Hit_and_Run',
           'Motor_Vehicle_Interacted_With',
           'Pedestrian_Action', 
           'Road_Condition_1',
           'Road_Condition_2',
           'Road_Surface',
           'Lighting',
           'Pedestrian_Involved',
           'Bicycle_Involved',
           'Motorcycle_Involved',
           'Alcohol_Involved',
           'Data_Source',
           'POINT_X',
           'POINT_Y']].copy()


Crash Feature Classes Merged
Started data transfer: 2023-12-08 11:49:51
Finished data transfer: 2023-12-08 11:49:56
Finished updating staging layer
features deleted


In [ ]:
#NV 2021 data transformation

# set fields for time and case info
dfNVCrash21['CA_Case_ID']           = np.nan
dfNVCrash21['NV_Accident_Num']      = dfNVCrash21['NV Accident Num']
dfNVCrash21['NV_Accident_Rec_Num']  = dfNVCrash21['NV Accident Rec Num']
dfNVCrash21['City']                 = np.nan
dfNVCrash21['Year']                 = dfNVCrash21['Collision_Year']
dfNVCrash21['Date']                 = dfNVCrash21['Collision_Date']
dfNVCrash21['Time']                 = dfNVCrash21['Collision_Time']
dfNVCrash21['Num_Vehicles']         = dfNVCrash21['Total Vehicles']
dfNVCrash21['Num_Parties']          = np.nan
dfNVCrash21['Data_Source']          = "NDOT"

# Convert NV crash type and severity
dfNVCrash21['Crash_Severity']       = dfNVCrash21['COLLISION_SEVERITY']
dfNVCrash21.loc[dfNVCrash21['VehCrashType'].isin(["ANGLE"]),  'Crash_Type']         = 'Angle-broadside'
dfNVCrash21.loc[dfNVCrash21['VehCrashType'].isin(["BACKING"]),  'Crash_Type']       = 'Backing'
dfNVCrash21.loc[dfNVCrash21['VehCrashType'].isin(["HEAD-ON"]),  'Crash_Type']       = 'Head-on'
dfNVCrash21.loc[dfNVCrash21['VehCrashType'].isin(["HEAD ON"]),  'Crash_Type']       = 'Head-on'
dfNVCrash21.loc[dfNVCrash21['VehCrashType'].isin(["NON-COLLISION"]),  'Crash_Type'] = 'Non-collision'
dfNVCrash21.loc[dfNVCrash21['VehCrashType'].isin(["REAR-END"]),  'Crash_Type']      = 'Rear end'
dfNVCrash21.loc[dfNVCrash21['VehCrashType'].isin(["REAR-TO-REAR"]),  'Crash_Type']  = 'Other'
dfNVCrash21.loc[dfNVCrash21['VehCrashType'].isin(["SIDESWIPE, MEETING"]),  'Crash_Type'] = 'Sideswipe'
dfNVCrash21.loc[dfNVCrash21['VehCrashType'].isin(["SIDESWIPE - MEETING"]),  'Crash_Type'] = 'Sideswipe'
dfNVCrash21.loc[dfNVCrash21['VehCrashType'].isin(["SIDESWIPE, OVERTAKING"]),  'Crash_Type'] = 'Sideswipe'
dfNVCrash21.loc[dfNVCrash21['VehCrashType'].isin(["SIDESWIPE - OVERTAKING"]),  'Crash_Type'] = 'Sideswipe'
dfNVCrash21.loc[dfNVCrash21['VehCrashType'].isin(["UNKNOWN"]),  'Crash_Type'] = 'Unknown'
#Convert # injured/killed
dfNVCrash21['Num_Killed']   = dfNVCrash21['NumFatalities']
dfNVCrash21['Num_Killed'].fillna(0)
dfNVCrash21['Num_Injured'] = dfNVCrash21['Injured']
dfNVCrash21['Num_Injured'].fillna(0)
dfNVCrash21['Num_Ped_Killed'] = np.nan
dfNVCrash21['Num_Ped_Injured'] = np.nan
dfNVCrash21['Num_Bicyclist_Killed'] = np.nan
dfNVCrash21['Num_Bicyclist_Injured'] = np.nan
dfNVCrash21['Num_Motorcyclist_Killed'] = np.nan
dfNVCrash21['Num_Motorcyclist_Injured'] = np.nan
# convert crash info
dfNVCrash21['Violation']  = "N/A"
dfNVCrash21['Hit_and_Run'] = "N/A"
dfNVCrash21['Motor_Vehicle_Interacted_With'] = "N/A"
dfNVCrash21['Pedestrian_Action'] = "N/A"
# Process lighting
dfNVCrash21['Lighting'] = dfNVCrash21['LightCondition']
dfNVCrash21.loc[dfNVCrash21['Lighting'].isin(["Dusk"]),  'Lighting'] = "Dusk - Dawn"
dfNVCrash21.loc[dfNVCrash21['Lighting'].isin(["Dawn"]),  'Lighting'] = "Dusk - Dawn"
dfNVCrash21.loc[dfNVCrash21['Lighting'].isin(["Dark - street lights"]),  'Lighting'] = "Dark - Street Lights"
dfNVCrash21.loc[dfNVCrash21['Lighting'].isin(["DARK - NO ROADWAY LIGHTING"]),  'Lighting'] = "Dark - No Street Lights"
dfNVCrash21.loc[dfNVCrash21['Lighting'].isin(["Dark - Unknown Lighting"]),  'Lighting'] = "Dark - Unknown Lighting"
dfNVCrash21.loc[dfNVCrash21['Lighting'].isin(["DAYLIGHT"]),  'Lighting'] = "Daylight"
dfNVCrash21.loc[dfNVCrash21['Lighting'].isin(["Unknown"]),  'Lighting'] = "Not Stated"
#Process alcohol involvement (check outputs)
dfNVCrash21['V1 Driver Factors'] = dfNVCrash21['V1 Driver Factors'].fillna("Not stated")
dfNVCrash21['V2 Driver Factors'] = dfNVCrash21['V2 Driver Factors'].fillna("Not stated")
dfNVCrash21.loc[dfNVCrash21['V1 Driver Factors'].str.contains("DRINKING"), 'Alcohol_Involved'] = "Y"
dfNVCrash21.loc[dfNVCrash21['V2 Driver Factors'].str.contains("DRINKING"), 'Alcohol_Involved'] = "Y"
#Process bike/ped involvement (check outputs)
dfNVCrash21['V1 All Events'] = dfNVCrash21['V1 All Events'].fillna("Not stated")
dfNVCrash21['V2 All Events'] = dfNVCrash21['V2 All Events'].fillna("Not stated")
dfNVCrash21.loc[dfNVCrash21['V1 All Events'].str.contains("PEDESTRIAN"), 'Pedestrian_Involved'] = "Y"
dfNVCrash21.loc[dfNVCrash21['V2 All Events'].str.contains("PEDESTRIAN"), 'Pedestrian_Involved'] = "Y"
dfNVCrash21.loc[dfNVCrash21['V1 All Events'].str.contains("PEDAL CYCLE"), 'Bicycle_Involved'] = "Y"
dfNVCrash21.loc[dfNVCrash21['V2 All Events'].str.contains("PEDAL CYCLE"), 'Bicycle_Involved'] = "Y"
#Process motorcycle involvement (check outputs)
dfNVCrash21['V1 Type'] = dfNVCrash21['V1 Type'].fillna("Not stated")
dfNVCrash21['V2 Type'] = dfNVCrash21['V1 Type'].fillna("Not stated")
dfNVCrash21.loc[dfNVCrash21['V1 Type'].str.contains("MOTORCYCLE"), 'Motorcycle_Involved'] = "Y"
dfNVCrash21.loc[dfNVCrash21['V1 Type'].str.contains("MOTORBIKE"), 'Motorcycle_Involved'] = "Y"
dfNVCrash21.loc[dfNVCrash21['V1 Type'].str.contains("MOPED"), 'Motorcycle_Involved'] = "Y"
dfNVCrash21.loc[dfNVCrash21['V2 Type'].str.contains("MOTORCYCLE"), 'Motorcycle_Involved'] = "Y"
dfNVCrash21.loc[dfNVCrash21['V2 Type'].str.contains("MOTORBIKE"), 'Motorcycle_Involved'] = "Y"
dfNVCrash21.loc[dfNVCrash21['V2 Type'].str.contains("MOPED"), 'Motorcycle_Involved'] = "Y"
# #Convert road surface
dfNVCrash21['Factors Roadway'] = dfNVCrash21['Factors Roadway'].fillna("Not stated")
dfNVCrash21['HWY Factors'] = dfNVCrash21['HWY Factors'].fillna("Not stated")
dfNVCrash21.loc[dfNVCrash21['Factors Roadway'].str.contains("DRY"), 'Road_Surface'] = "Dry"
dfNVCrash21.loc[dfNVCrash21['Factors Roadway'].str.contains("WET"), 'Road_Surface'] = "Wet"
dfNVCrash21.loc[dfNVCrash21['Factors Roadway'].str.contains("WATER"), 'Road_Surface'] = "Wet"
dfNVCrash21.loc[dfNVCrash21['Factors Roadway'].str.contains("ICE"), 'Road_Surface'] = "Snowy or Icy"
dfNVCrash21.loc[dfNVCrash21['Factors Roadway'].str.contains("SNOW"), 'Road_Surface'] = "Snowy or Icy"
dfNVCrash21.loc[dfNVCrash21['Factors Roadway'].str.contains("SLUSH"), 'Road_Surface'] = "Snowy or Icy"
dfNVCrash21.loc[dfNVCrash21['Factors Roadway'].str.contains("OTHER"), 'Road_Surface'] = "Other"
dfNVCrash21.loc[dfNVCrash21['Factors Roadway'].str.contains("NA"), 'Road_Surface'] = "Not stated"
dfNVCrash21.loc[dfNVCrash21['Factors Roadway'].str.contains("UNKNOWN"), 'Road_Surface'] = "Unknown"
dfNVCrash21.loc[dfNVCrash21['Factors Roadway'].str.contains("Not stated"), 'Road_Surface'] = "Not stated"
#Convert road condition
dfNVCrash21.loc[dfNVCrash21['RoadEnvironmentalFactors'].str.contains("NONE"), 'Road_Condition_1'] = "No Unusual Condition"
dfNVCrash21.loc[dfNVCrash21['RoadEnvironmentalFactors'].str.contains("UNKNOWN"), 'Road_Condition_1'] = "Unknown"
dfNVCrash21.loc[dfNVCrash21['RoadEnvironmentalFactors'].str.contains("WET, ICY, SNOW, SLUSH"), 'Road_Condition_1'] = "Weather"
dfNVCrash21.loc[dfNVCrash21['RoadEnvironmentalFactors'].str.contains("WEATHER"), 'Road_Condition_1'] = "Weather"
dfNVCrash21.loc[dfNVCrash21['RoadEnvironmentalFactors'].str.contains("BACKUP"), 'Road_Condition_1'] = "Other"
dfNVCrash21.loc[dfNVCrash21['RoadEnvironmentalFactors'].str.contains("GLARE"), 'Road_Condition_1'] = "Other"
dfNVCrash21.loc[dfNVCrash21['RoadEnvironmentalFactors'].str.contains("OTHER"), 'Road_Condition_1'] = "Other"
dfNVCrash21.loc[dfNVCrash21['RoadEnvironmentalFactors'].str.contains("VISUAL OBSTRUCTION"), 'Road_Condition_1'] = "Other"
dfNVCrash21.loc[dfNVCrash21['RoadEnvironmentalFactors'].str.contains("ROAD OBSTRUCTION"), 'Road_Condition_1'] = "Obstruction on Roadway"
dfNVCrash21.loc[dfNVCrash21['RoadEnvironmentalFactors'].str.contains("DEBRIS"), 'Road_Condition_1'] = "Loose Material on Roadway"
dfNVCrash21.loc[dfNVCrash21['RoadEnvironmentalFactors'].str.contains("WORK ZONE"), 'Road_Condition_1'] = "Construction or Repair Zone"
dfNVCrash21.loc[dfNVCrash21['RoadEnvironmentalFactors'].str.contains("ANIMAL"), 'Road_Condition_1'] = "Wildlife"
dfNVCrash21.loc[dfNVCrash21['RoadEnvironmentalFactors'].str.contains("RUTS"), 'Road_Condition_1'] = "Holes, Deep Ruts"
dfNVCrash21.loc[dfNVCrash21['RoadEnvironmentalFactors'].str.contains("Not stated"), 'Road_Condition_1'] = "Not Stated"
dfNVCrash21['Road_Condition_2'] = np.nan

#Rename to match CA data
dfNVCrash21['Weather_1']   = dfNVCrash21['Weather']
dfNVCrash21['Weather_2']   = np.nan
dfNVCrash21['POINT_X']     = dfNVCrash21['X']
dfNVCrash21['POINT_Y']     = dfNVCrash21['Y']
dfNVCrash21['Corridor_ID'] = np.nan

# final list of fields
dfNVCrash21 = dfNVCrash21[['State',
           'CA_Case_ID',
           'NV_Accident_Num',
           'NV_Accident_Rec_Num',
           'Corridor_ID',
           'County',
           'City',
           'Year',
           'Date',
           'Time',
           'Weather_1',
           'Weather_2',
           'Crash_Severity',
           'Num_Killed',
           'Num_Injured',
           'Num_Ped_Killed',
           'Num_Ped_Injured',
           'Num_Bicyclist_Killed',
           'Num_Bicyclist_Injured',
           'Crash_Type',
           'Num_Vehicles',
           'Num_Parties',
           'Violation',
           'Hit_and_Run',
           'Motor_Vehicle_Interacted_With',
           'Pedestrian_Action', 
           'Road_Condition_1',
           'Road_Condition_2',
           'Road_Surface',
           'Lighting',
           'Pedestrian_Involved',
           'Bicycle_Involved',
           'Motorcycle_Involved',
           'Alcohol_Involved',
           'Data_Source',
           'POINT_X',
           'POINT_Y']].copy()

In [ ]:

# os.remove(os.path.join(workspace, "NV_Crash_New.csv" ))

# export dataframe to csv 
dfNVCrash.to_csv(os.path.join(workspace, "NV_Crash_New.csv" ))
# get NV CSV for XY Table TO Point
nvCSV = os.path.join(workspace, "NV_Crash_New.csv" )

# name the output feature class
nvFC  = 'NV_Crash_New'



# Nevada data frame to feature class 
# input data is in NAD 1983 UTM Zone 11N coordinate system
arcpy.management.XYTableToPoint(nvCSV, nvFC, 
                                x_coords, y_coords, "",
                                # set prjoection transform to from
                                arcpy.SpatialReference(26911))

# output data for project tool
output_NV_Crash_Project = "NV_Crash_Project"

# project from UTM to WGS
arcpy.Project_management(nvFC, output_NV_Crash_Project, out_coordinate_system)

# os.remove(os.path.join(workspace, "CA_Crash_New.csv"))
## CA Export
# export dataframe to csv 
dfCACrash.to_csv(os.path.join(workspace, "CA_Crash_New.csv" ))

# get NV CSV for XY Table TO Point
caCSV = os.path.join(workspace, "CA_Crash_New.csv" )

# name the output feature class
caFC     = 'CA_Crash_New'

# CA data frame to feature class
arcpy.management.XYTableToPoint(caCSV, caFC, 
                                x_coords, y_coords, "",
                                # set prjoection transform to from
                                arcpy.SpatialReference(4326))

# output data for project tool
output_CA_Crash_Project = "CA_Crash_Project" 

# project from UTM to WGS
arcpy.Project_management(caFC, output_CA_Crash_Project, out_coordinate_system)

## Merge CA and NV
# out merge fc
tahoeCrash = "Tahoe_Crash"

# input feature classes
caCrash = "CA_Crash_Project"
nvCrash = "NV_Crash_Project"

# Create FieldMappings object to manage merge output fields
fieldMappings = arcpy.FieldMappings()
# Add all fields from all parcel staging layers
fieldMappings.addTable(caCrash)
fieldMappings.addTable(nvCrash)

# Remove all output fields from the field mappings, except fields in field_master list
for field in fieldMappings.fields:
    if field.name not in [  'OBJECTID',
                            'State',
                            'CA_Case_ID',
                            'NV_Accident_Num',
                            'NV_Accident_Rec_Num',
                            'Corridor_ID',
                            'County',
                            'City',
                            'Year',
                            'Date',
                            'Time',
                            'Weather_1',
                            'Weather_2',
                            'Crash_Severity',
                            'Num_Killed',
                            'Num_Injured',
                            'Num_Ped_Killed',
                            'Num_Ped_Injured',
                            'Num_Bicyclist_Killed',
                            'Num_Bicyclist_Injured',
                            'Num_Motorcyclist_Killed',
                            'Num_Motorcyclist_Injured',
                            'Crash_Type',
                            'Num_Vehicles',
                            'Num_Parties',
                            'Violation',
                            'Hit_and_Run',
                            'Motor_Vehicle_Interacted_With',
                            'Pedestrian_Action', 
                            'Road_Condition_1',
                            'Road_Condition_2',
                            'Road_Surface',
                            'Lighting',
                            'Pedestrian_Involved',
                            'Bicycle_Involved',
                            'Alcohol_Involved',
                            'Motorcycle_Involved',
                            'Data_Source',
                            'POINT_X',
                            'POINT_Y',
                            'SHAPE@']:
        # remove everything else
        fieldMappings.removeFieldMap(fieldMappings.findFieldMapIndex(field.name)) 
    
# Use Merge tool to move features into single dataset
arcpy.management.Merge([caCrash, nvCrash], tahoeCrash, fieldMappings)
print("Crash Feature Classes Merged")

# ## Spatial Join of Corridor IDs
# in memory points to be used for spatial join results
corridorPoints = memory + 'CrashPoint_Corridor'
# Spatial Join
arcpy.SpatialJoin_analysis(tahoeCrash, corridor, corridorPoints, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "HAVE_THEIR_CENTER_IN", "", "")

# use function to transfer spatial join results to crash stagin layer
fieldJoinCalc(tahoeCrash, ['OBJECTID', 'Corridor_ID'], corridorPoints, ['OBJECTID','CORRIDOR_NAME'])
print("Finished updating staging layer")


tempLayer = "deleteLayers"

# Run MakeFeatureLayer
arcpy.management.MakeFeatureLayer(tahoeCrash, tempLayer)
 
arcpy.management.SelectLayerByLocation(tempLayer, "have_their_center_in", 
                                       trpa,
                                       search_distance="", 
                                       selection_type="NEW_SELECTION", 
                                       invert_spatial_relationship="INVERT")
 
# Run GetCount and if some features have been selected, then 
#  run DeleteFeatures to remove the selected features.
if int(arcpy.management.GetCount(tempLayer)[0]) > 0:
    arcpy.management.DeleteFeatures(tempLayer)
print("features deleted")

# outfc = 
# Update SDE - Truncate Append
# updateSDE(tahoeCrash, outfc, fieldnames)

### Data Export 

### Photo Export

In [ ]:
import arcpy
from arcpy import da
import os
# setup workspace folder
workspace = "//Trpa-fs01/gis/GIS_DATA/Monitoring/StreamBioassessment/StreamBioassessment_Survey/Survey_Photos/2022"


# arcpy.env.workspace = "C:\GIS\Scratch.gdb"

# network path to connection files
filePath = "C:\\GIS\\DB_CONNECT"

# database file path 
sdeBase = os.path.join(filePath,"Collection.sde")

photoTable = os.path.join(sdeBase, 'sde_collection.SDE.StreamBio_Photo_2022__ATTACH')

with da.SearchCursor(photoTable, ['DATA', 'ATT_NAME', 'ATTACHMENTID']) as cursor:
    for item in cursor:
        attachment = item[0]
        filenum = "ATT" + str(item[2]) + "_"
        filename = filenum + str(item[1])
        open(workspace + os.sep + filename, 'wb').write(attachment.tobytes())
        print(filename)
        del item
        del filenum
        del filename
        del attachment

## Bike Ped Data

### How to download the data

* __Website:__ [Trafx Data](https://www.trafx.net/datanet/login)
__username__: stevlin@trpa.org
__password__: _CampOrkila_

__TRT Login__
__username__: info@tahoerimtrail.org
__password__: _Trta8782!_

> To get trafx data, hit the main page, then:
 - go to the analyze tab 
 - select all sites and all date ranges 
 - select daily counts 
 - download csv and enter csv name below

* __Website:__ [Ecovisio](https://www.eco-visio.net/Ecovisio/)
__username__: _trpc.admin_
__password__: _bikecount_

> To get EcoVisio data: go to this [page](https://www.eco-visio.net/v5/#analysis::module=6), then:
 - hit the 'select all' text above the teal 'new counters' bar 
 - press analyze selection in top left
 - under the period bar, select entire period
 - hit the spreadsheet symbol then click the spreadsheet option
 - hit the export symbol and chose csv
 - chose the default export options
 
 
 * __Save__ the file to the [workspace](//Trpa-fs01/gis/PROJECTS/ResearchAnalysis/Monitoring/Data/BikePed/)




### Import Modules

In [1]:
# import modules
import pandas as pd
import os
import pyodbc
from datetime import datetime
import sqlalchemy
import pyodbc
import arcpy
from arcgis.features import FeatureSet, GeoAccessor, GeoSeriesAccessor

# setup workspace folder
workspace = "//Trpa-fs01/gis/PROJECTS/ResearchAnalysis/Monitoring/Data/BikePed/"


arcpy.env.workspace = "C:\GIS\Scratch.gdb"

# network path to connection files
filePath = "C:\\GIS\\DB_CONNECT"

# database file path 
sdeBase = os.path.join(filePath, "Vector.sde")


In [2]:
# make sql database connection to BMP with pyodbc
sdeTabular = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=sql12;DATABASE=sde_tabular;UID=sde;PWD=staff')

# Final table to update
outTable = "sde_tabular.SDE.bike_ped_counter_tabular"

### Get Current Bike Ped Data

In [3]:
# Parcels - create a dataframe from SDE Parcel Master
counters = sdeBase + "\\sde.SDE.Transportation\\sde.SDE.bike_ped_counter_spatial"
sdfBike = pd.DataFrame.spatial.from_featureclass(counters)

In [4]:
sdfBike.counter_id.unique()

array(['counter_13', 'counter_16', 'counter_17', 'counter_18',
       'counter_19', 'counter_20', 'counter_21', 'counter_25',
       'counter_27', 'counter_28', 'counter_29', 'counter_32',
       'counter_33', 'counter_37', 'counter_38', 'counter_39',
       'counter_40', 'counter_43', 'counter_44', 'counter_30',
       'counter_36', 'counter_41', 'counter_42', 'counter_1', 'counter_2',
       'counter_3', 'counter_4', 'counter_5', 'counter_6', 'counter_7',
       'counter_8', 'counter_9', 'counter_10', 'counter_11', 'counter_12',
       'counter_14', 'counter_15', 'counter_22', 'counter_24',
       'counter_26', 'counter_34', 'counter_35', 'counter_45',
       'counter_46', 'counter_47'], dtype=object)

In [5]:
# Get Bike Ped SQL Table
dfBike      = pd.read_sql("SELECT * FROM sde_tabular.SDE.bike_ped_counter_tabular", sdeTabular)
dfBike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30390 entries, 0 to 30389
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   OBJECTID           30390 non-null  int64 
 1   month_day_year     30390 non-null  object
 2   counter_name       30390 non-null  object
 3   month_of_year      30390 non-null  int64 
 4   season_of_year     30390 non-null  object
 5   count_of_bike_ped  30390 non-null  int64 
 6   counter_category   30390 non-null  object
 7   counter_id         28005 non-null  object
dtypes: int64(3), object(5)
memory usage: 1.9+ MB


In [6]:
# export OG data to workspace
dfBike.to_csv(os.path.join(workspace,"BikePed_Original.csv"))

In [7]:
dfBike = pd.read_csv(os.path.join(workspace,"BikePed_Original.csv"))

In [8]:
dfBike.month_day_year.min()

'1/1/2017'

In [9]:
n = len(dfBike.counter_name.unique())

print("number of counters:",n)

number of counters: 44


In [10]:
# create dictionary of counter_name:counter_id to be used to assign counter_id to the new data
counterDict = sdfBike.set_index('counter_name').to_dict()['counter_id']

In [11]:
for key,value in counterDict.items():
    print("{} : {}".format(key,value))

Shared-use path - Baldwin Beach : counter_13
Shared-use path - Lake Forest : counter_16
Shared-use path - Lakeshore Blvd : counter_17
Shared-use path - Lakeside Trail : counter_18
Shared-use path - Lakeview Commons : counter_19
Shared-use path - Linear Park : counter_20
Shared-use path - Pinedrops : counter_21
Shared-use path - Ski Run Blvd : counter_25
Shared-use path - South Tahoe Bikeway : counter_27
Shared-use path - Sugar Pine Point  : counter_28
Shared-use path - Sunnyside : counter_29
Shared-use path - US 50 at Pioneer (Stateline) : counter_32
Shared-use path - Zephyr Cove : counter_33
Sidewalk - Carnelian Woods Avenue : counter_37
Sidewalk - Emerald Bay : counter_38
Sidewalk - Kings Beach Recreation Area : counter_39
Sidewalk - Kingsbury Grade : counter_40
Sidewalk - US 50 at Lake Pkwy : counter_43
Sidewalk - Village Blvd : counter_44
Shared-use path - Trout Creek bridge : counter_30
Sidewalk - Al Tahoe Blvd : counter_36
Sidewalk - Pioneer Trail at Meyers : counter_41
Sidewalk 

### Transform Trafx Data

In [12]:
trafxTRTA2021 = pd.read_csv(os.path.join(workspace, "trafx_daily_trta_2021.csv"))

In [14]:
trafxTRTA2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611 entries, 0 to 610
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Day                           611 non-null    object 
 1   2021 Barker North Bike        0 non-null      float64
 2   2021 Barker North Bike.1      144 non-null    float64
 3   2021 Before Picnic Rock       136 non-null    float64
 4   2021 Brockway Summit East     116 non-null    float64
 5   2021 Brockway Summit West     116 non-null    float64
 6   2021 Bryan Meadow South Bike  121 non-null    float64
 7   2021 Bryan North Bike         126 non-null    float64
 8   2021 Echo Lake                129 non-null    float64
 9   2021 Genoa Peak Rd            136 non-null    float64
 10  2021 Meiss Bike               122 non-null    float64
 11  2021 Mt. Rose Gal. Falls      117 non-null    float64
 12  2021 Scotts Lake              114 non-null    float64
 13  2021 

In [15]:
trafxTRTA2021.drop(trafxTRTA2021.columns[[25,26,27,28]], axis=1, inplace=True)

IndexError: index 25 is out of bounds for axis 0 with size 19

In [16]:
# get a list of the fields that will be values
dfCol = trafxTRTA2021.iloc[:,2:18]
trafxList = dfCol.columns.to_list()

In [17]:
# transfrom data from wide to long format
dfTrafx = pd.melt(trafxTRTA2021.reset_index(), id_vars = ['Day'], value_vars = trafxList)

In [18]:
# change the format of field to Date
dfTrafx['Day'] = pd.to_datetime(dfTrafx['Day'])
# change the format to MM-DD-YYYY
dfTrafx['Day'] = dfTrafx['Day'].dt.strftime('%m-%d-%Y')

In [19]:
dfTrafx = dfTrafx.rename(columns={'Day':'month_day_year', 
                        'variable':'counter_name', 
                        'value':'count_of_bike_ped'})

In [20]:
dfTrafx

,month_day_year,counter_name,count_of_bike_ped
0,06-14-2020,2021 Barker North Bike.1,NaN
1,06-15-2020,2021 Barker North Bike.1,NaN
2,06-16-2020,2021 Barker North Bike.1,NaN
3,06-17-2020,2021 Barker North Bike.1,NaN
4,06-18-2020,2021 Barker North Bike.1,NaN
...,...,...,...
9771,02-10-2022,2021 Spooner South,0.0
9772,02-11-2022,2021 Spooner South,0.0
9773,02-12-2022,2021 Spooner South,0.0
9774,02-13-2022,2021 Spooner South,114.0


In [26]:
# new


In [13]:
dfTrafx.counter_name = dfTrafx.counter_name.str.lstrip("2021 ")
dfTrafx.loc[dfTrafx["counter_name"]=="Barker North Bike.1", 'counter_name']="Barker North Bike"

NameError: name 'dfTrafx' is not defined

In [27]:
dfTrafx

,month_day_year,counter_name,count_of_bike_ped
0,06-14-2020,Barker North Bike,NaN
1,06-15-2020,Barker North Bike,NaN
2,06-16-2020,Barker North Bike,NaN
3,06-17-2020,Barker North Bike,NaN
4,06-18-2020,Barker North Bike,NaN
...,...,...,...
9771,02-10-2022,Spooner South,0.0
9772,02-11-2022,Spooner South,0.0
9773,02-12-2022,Spooner South,0.0
9774,02-13-2022,Spooner South,114.0


In [12]:
trafx = pd.read_csv(os.path.join(workspace, "trafx_daily.csv"))

In [21]:
trafx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1966 entries, 0 to 1965
Data columns (total 29 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Day                                                   1966 non-null   object 
 1   Shared-use path - Baldwin Beach                       781 non-null    float64
 2   Shared-use path - Lake Forest                         1131 non-null   float64
 3   Shared-use path - Lakeshore Blvd                      1030 non-null   float64
 4   Shared-use path - Lakeside Trail                      427 non-null    float64
 5   Shared-use path - Lakeview Commons                    1189 non-null   float64
 6   Shared-use path - Linear Park                         982 non-null    float64
 7   Shared-use path - Pinedrops                           740 non-null    float64
 8   Shared-use path - RoundHill                           313 

### Transform Trafx Data

In [14]:
trafx.drop(trafx.columns[[25,26,27,28]], axis=1, inplace=True)

NameError: name 'trafx' is not defined

In [23]:
# get a list of the fields that will be values
dfCol = trafx.iloc[:,1:24]
trafxList = dfCol.columns.to_list()

In [24]:
# transfrom data from wide to long format
dfTrafx = pd.melt(trafx.reset_index(), id_vars = ['Day'], value_vars = trafxList)

In [25]:
# change the format of field to Date
dfTrafx['Day'] = pd.to_datetime(dfTrafx['Day'])
# change the format to MM-DD-YYYY
dfTrafx['Day'] = dfTrafx['Day'].dt.strftime('%m-%d-%Y')

In [26]:
dfTrafx = dfTrafx.rename(columns={'Day':'month_day_year', 
                        'variable':'counter_name', 
                        'value':'count_of_bike_ped'})

### Transform EcoVision Data

In [27]:
eco = pd.read_csv(os.path.join(workspace, "ecovision_daily.csv"))

In [28]:
eco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2002 entries, 0 to 2001
Data columns (total 26 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Time                                            2002 non-null   object 
 1   East Shore trail - Hidden Beach                 980 non-null    float64
 2   East Shore trail - Incline Village              916 non-null    float64
 3   floating pneumatic tube - for editing           2 non-null      float64
 4   Pneumatic Tube - Carnelian Woods Ave.           14 non-null     float64
 5   Pneumatic Tube - Fallen Leaf Rd                 8 non-null      float64
 6   Pneumatic tube - Pioneer Trail at Sierra House  17 non-null     float64
 7   Pneumatic tube - Sierra Blvd                    4 non-null      float64
 8   Pneumatic Tube - US 50 at Al Tahoe Blvd         13 non-null     float64
 9   Pneumatic Tube - US 50 at Ski Run Blvd   

In [29]:
# drop columns
eco.drop(eco.columns[[25]], axis=1, inplace=True)

In [30]:
# get a list of the fields that will be values
dfCol = eco.iloc[:,1:24]
ecoList = dfCol.columns.to_list()

In [31]:
# transfrom data from wide to long format
dfEco = pd.melt(eco.reset_index(), id_vars = ['Time'], value_vars = ecoList)

In [32]:
# change the format of field to Date
dfEco['Time'] = pd.to_datetime(dfEco['Time'])
# change the format to MM-DD-YYYY
dfEco['Time'] = dfEco['Time'].dt.strftime('%m-%d-%Y')

In [33]:
dfEco = dfEco.rename(columns={'Time':'month_day_year', 
                        'variable':'counter_name', 
                        'value':'count_of_bike_ped'})

### Merge and Clean Up

In [34]:
# combine data frames
df = pd.concat([dfEco, dfTrafx])

In [28]:
df = dfTrafx

In [29]:
# drop NaN values
df.dropna(subset = ['count_of_bike_ped'],inplace=True)

In [30]:
# create the month field and set it's type
df.insert(loc=3,column='month_of_year', value = pd.to_datetime(df['month_day_year']).dt.month)
# calculate the values
df['month_of_year'] = pd.to_datetime(df['month_day_year']).dt.month

In [31]:
# set the season field values
df.loc[df['month_of_year'].isin([12,1,2,3]),  'season_of_year'] = 'Winter' 
df.loc[df['month_of_year'].isin([6,7,8,9]),   'season_of_year'] = 'Summer' 
df.loc[df['month_of_year'].isin([4,5,10,11]), 'season_of_year'] = 'Off-Season' 

In [33]:
df['counter_category']= 'TRTA Trafx'

In [38]:
# set the counter category values by comparing the list of station names created earliar
df.loc[df['counter_name'].isin(ecoList), 'counter_category']= 'ecovision'
df.loc[df['counter_name'].isin(trafxList), 'counter_category']= 'trafx'
df.loc[df['counter_name'].isin(ecoList) & df['counter_name'].isin(trafxList), 'counter_category']= 'ecovision & trafx'

* https://stackoverflow.com/questions/51881503/assign-a-dictionary-value-to-a-dataframe-column-based-on-dictionary-key


In [49]:
# create dictionary of counter_name:counter_id to be used to assign counter_id to the new data
counterDict = dfBike.set_index('counter_name').to_dict()['counter_id']

In [50]:
# set counter id based on counter_id in DF Bike using dictionary
df["counter_id"] = df["counter_name"].apply(lambda x: counterDict.get(x))

In [51]:
df.counter_id.unique()

array([None], dtype=object)

In [39]:
# new
df.loc[df["counter_name"]=="Shared-use path - Sierra Blvd", 'counter_id']="counter_44"
# new
df.loc[df["counter_name"]=='Shared-use path - Al Tahoe',     'counter_id']="counter_45"
# new
df.loc[df["counter_name"]=='Shared-use path - LTCC',         'counter_id']="counter_46"
# existing
df.loc[df["counter_name"]=='Shared-use path - Sawmill',      'counter_id']="counter_24"

In [36]:
df

,month_day_year,counter_name,count_of_bike_ped,month_of_year,season_of_year,counter_category
382,07-01-2021,Barker North Bike,1.0,7,Summer,TRTA Trafx
383,07-02-2021,Barker North Bike,5.0,7,Summer,TRTA Trafx
384,07-03-2021,Barker North Bike,2.0,7,Summer,TRTA Trafx
385,07-04-2021,Barker North Bike,4.0,7,Summer,TRTA Trafx
386,07-05-2021,Barker North Bike,3.0,7,Summer,TRTA Trafx
...,...,...,...,...,...,...
9770,02-09-2022,Spooner South,0.0,2,Winter,TRTA Trafx
9771,02-10-2022,Spooner South,0.0,2,Winter,TRTA Trafx
9772,02-11-2022,Spooner South,0.0,2,Winter,TRTA Trafx
9773,02-12-2022,Spooner South,0.0,2,Winter,TRTA Trafx


### Export

In [37]:
# save to CSV
df.to_csv(os.path.join(workspace,"BikePed_TRTA2021.csv"))

In [53]:
df['OBJECTID'] = df.reset_index().index
df.set_index('OBJECTID',inplace=True)

In [54]:
# get a list of the existing fields
bikeList = dfBike.columns.tolist()
del bikeList[0]
#reorder fields to match the sql table schema
dfNew = df[bikeList]

SyntaxError: invalid syntax (<ipython-input-54-2ab3c0d36b32>, line 4)

In [30]:
dfNew.info()

NameError: name 'dfNew' is not defined

In [48]:
# save to CSV
dfNew.to_csv(os.path.join(workspace,"BikePed_New.csv"))

In [57]:
# update sql table
dfNew.to_sql(outTable, engine, if_exists='replace', index = True)

In [45]:
dfNew.reset_index()

,index,OBJECTID,month_day_year,counter_name,month_of_year,season_of_year,count_of_bike_ped,counter_category,counter_id
0,1003,0,07-01-2019,East Shore trail - Hidden Beach,7,Summer,1874.0,ecovision,counter_1
1,1004,1,07-02-2019,East Shore trail - Hidden Beach,7,Summer,1970.0,ecovision,counter_1
2,1005,2,07-03-2019,East Shore trail - Hidden Beach,7,Summer,2143.0,ecovision,counter_1
3,1006,3,07-04-2019,East Shore trail - Hidden Beach,7,Summer,2534.0,ecovision,counter_1
4,1007,4,07-05-2019,East Shore trail - Hidden Beach,7,Summer,2944.0,ecovision,counter_1
...,...,...,...,...,...,...,...,...,...
29289,59413,29289,01-05-2022,Sidewalk - US 50 at Lake Pkwy,1,Winter,85.0,trafx,counter_43
29290,59414,29290,01-06-2022,Sidewalk - US 50 at Lake Pkwy,1,Winter,53.0,trafx,counter_43
29291,59415,29291,01-07-2022,Sidewalk - US 50 at Lake Pkwy,1,Winter,49.0,trafx,counter_43
29292,59416,29292,01-08-2022,Sidewalk - US 50 at Lake Pkwy,1,Winter,77.0,trafx,counter_43


#### Data check

In [199]:
dfNoID = df.loc[df['counter_id'] == 0]

In [201]:
dfNoID.counter_name.unique()

array(['Shared- use path - Sierra Blvd', 'Shared-use path - Al Tahoe',
       'Shared-use path - LTCC', 'Shared-use path - Sawmill',
       'Shared-use path - Truckee River Trail'], dtype=object)

In [ ]:
# spatial data frame of points
sdfBike = os.path

F:\GIS\DB_CONNECT\Vector.sde\sde.SDE.Transportation\sde.SDE.bike_ped_counter_spatial

### Production Script

In [ ]:
"""
ParcelTables_to_ParcelFeatures.py
Created: March 11th, 2022
Last Updated: March 11th, 2022
Mason Bindl, Tahoe Regional Planning Agency

This python script was developed to transform data from Trafx and Ecovision

This script uses Python 3.x and was designed to be used with 
the default ArcGIS Pro python enivorment "arcgispro-py3-clone", with
no need for installing new libraries.
"""
#--------------------------------------------------------------------------------------------------------#
# import packages and modules
import arcpy
from datetime import datetime
import os
import sys
import pyodbc
import pandas as pd
from arcgis.features import FeatureSet, GeoAccessor, GeoSeriesAccessor
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# set overwrite to true
arcpy.env.overwriteOutput = True

# in memory output file path
wk_memory = "memory" + "\\"

# set workspace and sde connections 
working_folder = "C:\GIS"
workspace = "//trpa-fs01/gis/PROJECTS/ResearchAnalysis/Monitoring/Data/BikePed/"
arcpy.env.workspace = "F:/gis/PROJECTS/ResearchAnalysis/Monitoring/Data/BikePed/"

# make sql database connection to BMP with pyodbc
sdeTabular = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=sql12;DATABASE=sde_tabular;UID=sde;PWD=staff')

# Final table to update
outTable = "sde_tabular.SDE.bike_ped_counter_tabular"

# start a timer for the entire script run
FIRSTstartTimer = datetime.now()

# Create and open log file.
complete_txt_path = os.path.join(working_folder, "BikePed_ETL_Log.txt")
print (complete_txt_path)
log = open(complete_txt_path, "w")

# Write results to txt file
log.write("Log: " + str(FIRSTstartTimer) + "\n")
log.write("\n")
log.write("Begin process:\n")
log.write("Process started at: " + str(FIRSTstartTimer) + "\n")
log.write("\n")

#---------------------------------------------------------------------------------------#
## GET DATA
#---------------------------------------------------------------------------------------#
# start timer for the get data requests
startTimer = datetime.now()

# Get Bike Ped SQL Table
dfBikeOG = pd.read_sql("SELECT * FROM sde_tabular.SDE.bike_ped_counter_tabular", sdeTabular)
# export OG data to workspace
dfBikeOG.to_csv(os.path.join(workspace,"BikePed_Original.csv"))

# read in CSV of OG data
dfBike = pd.read_csv(os.path.join(workspace,"BikePed_Original.csv"))

#read in Trafx data download
trafx = pd.read_csv(os.path.join(workspace, "trafx_daily.csv"))

# read in Ecovision data download
eco = pd.read_csv(os.path.join(workspace, "ecovision_daily.csv"))
       
# report how long it took to get the data
endTimer = datetime.now() - startTimer
print("\nTime it took to get the data: {}".format(endTimer))   
log.write("\nTime it took to get the data: {}".format(endTimer)) 
#---------------------------------------------------------------------------------------#
## Define Functions ##
#---------------------------------------------------------------------------------------#
##--------------------------------------------------------------------------------------------------------#
## SEND EMAIL WITH LOG FILE ##
##--------------------------------------------------------------------------------------------------------#
# path to text file
fileToSend = complete_txt_path

# email parameters
subject = "Bike/Ped ETL Log File"
sender_email = "infosys@trpa.org"
# password = ''
receiver_email = "gis@trpa.gov"

# send mail function
def send_mail(body):
    msg = MIMEMultipart()
    msg['Subject'] = subject
    msg['From'] = sender_email
    msg['To'] = receiver_email

    msgText = MIMEText('%s<br><br>Cheers,<br>GIS Team' % (body), 'html')
    msg.attach(msgText)

    attachment = MIMEText(open(fileToSend).read())
    attachment.add_header("Content-Disposition", "attachment", filename = os.path.basename(fileToSend))
    msg.attach(attachment)

    try:
        with smtplib.SMTP("mail.smtp2go.com", 25) as smtpObj:
            smtpObj.ehlo()
            smtpObj.starttls()
#             smtpObj.login(sender_email, password)
            smtpObj.sendmail(sender_email, receiver_email, msg.as_string())
    except Exception as e:
        print(e)

# replaces features in outfc with exact same schema
def updateSDE(inputfc,outfc, fieldnames):
        # disconnect all users
    print("\nDisconnecting all users...")
    arcpy.DisconnectUser(sde, "ALL")

    # deletes all rows from the SDE feature class
    arcpy.TruncateTable_management(outfc)
    print ("\nDeleted all records in: {}\n".format(outfc))

    # insert rows from Temporary feature class to SDE feature class
    with arcpy.da.InsertCursor(outfc, fieldnames) as oCursor:
        count = 0
        with arcpy.da.SearchCursor(inputfc, fieldnames) as iCursor:
            for row in iCursor:
                oCursor.insertRow(row)
                count += 1
                if count % 1000 == 0:
                    print("Inserting record {0} into SDE table".format(count))

    # disconnect all users
    print("\nDisconnecting all users...")
    arcpy.DisconnectUser(sde, "ALL")
    # confirm feature class was created
    print("\nUpdated " + outfc)

try:
    #---------------------------------------------------------------------------------------#
    ## CREATE STAGING TABLE ##
    #---------------------------------------------------------------------------------------#
    # start timer for the get data requests
    startTimer = datetime.now()
    #---------------------------------------------------------------------------------------#
    
    # drop fields
    trafx.drop(trafx.columns[[25,26,27,28]], axis=1, inplace=True)
    
    # get a list of the fields that will be values
    dfCol = trafx.iloc[:,1:24]
    trafxList = dfCol.columns.to_list()
    
    # transfrom data from wide to long format
    dfTrafx = pd.melt(trafx.reset_index(), id_vars = ['Day'], value_vars = trafxList)

    # change the format of field to Date
    dfTrafx['Day'] = pd.to_datetime(dfTrafx['Day'])
    # change the format to MM-DD-YYYY
    dfTrafx['Day'] = dfTrafx['Day'].dt.strftime('%m-%d-%Y')
    # rename fields
    dfTrafx = dfTrafx.rename(columns={'Day':'month_day_year', 
                        'variable':'counter_name', 
                        'value':'count_of_bike_ped'})

    # drop columns
    eco.drop(eco.columns[[25, 26]], axis=1, inplace=True)
    # get a list of the fields that will be values
    dfCol = eco.iloc[:,1:24]
    ecoList = dfCol.columns.to_list()
    # transfrom data from wide to long format
    dfEco = pd.melt(eco.reset_index(), id_vars = ['Time'], value_vars = ecoList)
    # change the format of field to Date
    dfEco['Time'] = pd.to_datetime(dfEco['Time'])
    # change the format to MM-DD-YYYY
    dfEco['Time'] = dfEco['Time'].dt.strftime('%m-%d-%Y')
    # rename fields
    dfEco = dfEco.rename(columns={'Time':'month_day_year', 
                        'variable':'counter_name', 
                        'value':'count_of_bike_ped'})

    # combine data frames
    df = pd.concat([dfEco, dfTrafx])
    # drop NaN values
    df.dropna(subset = ['count_of_bike_ped'], inplace=True)
    # create the month field and set it's type
    df.insert(loc=3,column='month_of_year', value = pd.to_datetime(df['month_day_year']).dt.month)
    # calculate the values
    df['month_of_year'] = pd.to_datetime(df['month_day_year']).dt.month

    # set the season field values
    df.loc[df['month_of_year'].isin([12,1,2,3]),  'season_of_year'] = 'Winter' 
    df.loc[df['month_of_year'].isin([6,7,8,9]),   'season_of_year'] = 'Summer' 
    df.loc[df['month_of_year'].isin([4,5,10,11]), 'season_of_year'] = 'Off-Season' 

    # set the counter category values by comparing the list of station names created earliar
    df.loc[df['counter_name'].isin(ecoList), 'counter_category']= 'ecovision'
    df.loc[df['counter_name'].isin(trafxList), 'counter_category']= 'trafx'
    df.loc[df['counter_name'].isin(ecoList) & df['counter_name'].isin(trafxList), 'counter_category']= 'ecovision & trafx'
    
    # create dictionary of counter_name:counter_id to be used to assign counter_id to the new data
    counterDict = dfBike.set_index('counter_name').to_dict()['counter_id']
    # set counter id based on counter_id in DF Bike using dictionary
    df["counter_id"] = df["counter_name"].apply(lambda x: counterDict.get(x))
    
    # set OBJECTID field to be the index
    df['OBJECTID'] = df.reset_index().index
    df.set_index('OBJECTID',inplace=True)

    # save to CSV
    df.to_csv(os.path.join(workspace,"BikePed_New.csv"))

    #sde connection to disconnect users
    sde = "C:\\GIS\\DB_CONNECT\\Tabular.sde"

    # Change this to the path of your input feature class
    inputfc = os.path.join(workspace,"BikePed_New.csv")

    # Change this to the path of your output FC
    outfc = os.path.join(sde,"sde_tabular.SDE.bike_ped_counter_tabular")

    # Get field objects from source FC
    dsc = arcpy.Describe(inputfc)
    fields = dsc.fields

    # # List all field names except the OID field
    fieldnames = [field.name for field in fields if field.name != "Field1"]

    updateSDE(inputfc, outfc, fieldnames)

    # report how long it took to run the script
    endTimer = datetime.now() - startTimer
    print ("\nTime it took to run this script: {}".format(endTimer))
 
    # report how long it took to run the script
    FINALendTimer = datetime.now() - FIRSTstartTimer
    print ("\nTime it took to run this script: {}".format(FINALendTimer))

    log.write("\nTime it took to run this script: {}".format(FINALendTimer))
    log.close()
    
    header = "SUCCESS - The Bike/Ped data was updated."
    # send email with header based on try/except result
    send_mail(header)
    print('Sending email...')

# catch any arcpy errors
except arcpy.ExecuteError:
    print(arcpy.GetMessages())
    log.write(arcpy.GetMessages())
    log.close()
    
    header = "ERROR - Arcpy Exception - Check Log"
    # send email with header based on try/except result
    send_mail(header)
    print('Sending email...')

# catch system errors
except Exception:
    e = sys.exc_info()[1]
    print(e.args[0])
    log.write(str(e))
    log.close()
    
    header = "ERROR - System Error - Check Log"
    # send email with header based on try/except result
    send_mail(header)
    print('Sending email...')
